In this notebook we are walking through the data analysis example of chapter 14 from the book Python for Data Analysis by Wes McKinney from 2018.
The original code example can be found here: https://github.com/wesm/pydata-book/blob/2nd-edition/ch14.ipynb

**Instead of using Python pandas for the data analysis, we replicate the data analysis with Pyspark.** The corresponsing pandas notebook you can find here: https://www.kaggle.com/christine12/movielens-1m-dataset-python-pandas


In [10]:
#%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [12]:
spark = SparkSession.Builder().getOrCreate()

In [13]:
spark

# Import data

Dataset contains user rating data for movies collected from 1990s - 2000s.

movie ratings
movie meta data like genres and year
demographics of users age, zip code, gender identification, and occupation
An application for this kind of data could be recommendation systems

Import the 3 different files. In kaggle we need to adapt the orginal path to the data and specify the loading engine for read_table() to python to avoid warning messages.

Import the 3 different files. In kaggle we need to adapt the orginal path to the data and specify the loading engine for read_table() to python to avoid warning messages.

In [14]:
from pyspark.sql.functions import col

"""
Reading csv data in spark:

spark.read                         --> what do you want to do
    .format("csv")                 --> which format is the file
    .option("delimiter", "::")     --> specify delimiter that's used in the file
    .option("inferSchema", "true") --> values are read as string type by default
    .load(path_to_file)            --> which location to load the file from

"""
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users_sdf = spark.read.format("csv").option("delimiter", "::").option("inferSchema", "true").load("DATA SOURCE/users.dat")
u_cols = users_sdf.columns
users_sdf = users_sdf.select([col(u_cols[i]).alias(unames[i]) for i in range(len(unames))])

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_sdf = spark.read.format("csv").option("delimiter", "::").option("inferSchema", "true").load("DATA SOURCE/ratings.dat")
ratings_sdf = ratings_sdf.select([col(u_cols[i]).alias(rnames[i]) for i in range(len(rnames))])

mnames = ['movie_id', 'title', 'genres']
movies_sdf = spark.read.format("csv").option("delimiter", "::").option("inferSchema", "true").load("DATA SOURCE/movies.dat")
movies_sdf = movies_sdf.select([col(u_cols[i]).alias(mnames[i]) for i in range(len(mnames))])


Now we can varify that the input data was loaded correcly, by having a look at the first rows of each file. 

In [15]:
users_sdf.take(5)

[Row(user_id=1, gender='F', age=1, occupation=10, zip='48067'),
 Row(user_id=2, gender='M', age=56, occupation=16, zip='70072'),
 Row(user_id=3, gender='M', age=25, occupation=15, zip='55117'),
 Row(user_id=4, gender='M', age=45, occupation=7, zip='02460'),
 Row(user_id=5, gender='M', age=25, occupation=20, zip='55455')]

In [16]:
users_sdf.show(5)

+-------+------+---+----------+-----+
|user_id|gender|age|occupation|  zip|
+-------+------+---+----------+-----+
|      1|     F|  1|        10|48067|
|      2|     M| 56|        16|70072|
|      3|     M| 25|        15|55117|
|      4|     M| 45|         7|02460|
|      5|     M| 25|        20|55455|
+-------+------+---+----------+-----+


In [17]:
users_sdf.dtypes

[('user_id', 'int'),
 ('gender', 'string'),
 ('age', 'int'),
 ('occupation', 'int'),
 ('zip', 'string')]

Note: Ages and occupation are encoded as integers.

In [18]:
ratings_sdf.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|    1193|     5|978300760|
|      1|     661|     3|978302109|
|      1|     914|     3|978301968|
|      1|    3408|     4|978300275|
|      1|    2355|     5|978824291|
+-------+--------+------+---------+


Movie genre is a pipe-separated string, so analysis of genre require some transformation to bring this column in a usable form. 

In [19]:
movies_sdf.show(5)

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Animation|Childre...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|        Comedy|Drama|
|       5|Father of the Bri...|              Comedy|
+--------+--------------------+--------------------+


We want to analyse the mean ratings for a specific movie by age and gender. Next we need to merge all data into one dataframe to make handling easier. Merging strategy is to first merge ratings with users and followed by merging movies. Pandas infers which columns to use for the merge by finding overlapping names.

In [20]:
data_sdf = ratings_sdf.join(users_sdf, ["user_id"]).join(movies_sdf, ["movie_id"])


Let's take a look at the merged data frame. 

In [21]:
data_sdf.show(5)

+--------+-------+------+---------+------+---+----------+-----+--------------------+--------------------+
|movie_id|user_id|rating|timestamp|gender|age|occupation|  zip|               title|              genres|
+--------+-------+------+---------+------+---+----------+-----+--------------------+--------------------+
|    1193|      1|     5|978300760|     F|  1|        10|48067|One Flew Over the...|               Drama|
|     661|      1|     3|978302109|     F|  1|        10|48067|James and the Gia...|Animation|Childre...|
|     914|      1|     3|978301968|     F|  1|        10|48067| My Fair Lady (1964)|     Musical|Romance|
|    3408|      1|     4|978300275|     F|  1|        10|48067|Erin Brockovich (...|               Drama|
|    2355|      1|     5|978824291|     F|  1|        10|48067|Bug's Life, A (1998)|Animation|Childre...|
+--------+-------+------+---------+------+---+----------+-----+--------------------+--------------------+


# Calculate mean movie ratings for each film by gender using the pivot table method

The function pivot table reshapes the data frame depending on index and columns from input parameters. 
https://databricks.com/blog/2016/02/09/reshaping-data-with-pivot-in-apache-spark.html

We create a new data frame with mean ratings for each move title and split gender as columns (F and M).  


In [22]:
data_sdf.columns

['movie_id',
 'user_id',
 'rating',
 'timestamp',
 'gender',
 'age',
 'occupation',
 'zip',
 'title',
 'genres']

In [23]:
mean_ratings_pivot = data_sdf.groupBy("title").pivot("gender").agg(mean("rating"))

In [24]:
mean_ratings_pivot.sort('title').show(5)

+--------------------+------------------+------------------+
|               title|                 F|                 M|
+--------------------+------------------+------------------+
|$1,000,000 Duck (...|             3.375| 2.761904761904762|
|'Night Mother (1986)| 3.388888888888889|3.3529411764705883|
|'Til There Was Yo...| 2.675675675675676|2.7333333333333334|
|  'burbs, The (1989)|2.7934782608695654| 2.962085308056872|
|...And Justice fo...|3.8285714285714287|3.6890243902439024|
+--------------------+------------------+------------------+


Filter movies with low rating (less than 250).
* group data by title and get size() 
* extract list of titles with ratings by title >= 250
* use list to select mean ratings


In [25]:
ratings_by_title_sdf = data_sdf.groupBy("title").count()
ratings_by_title_sdf.sort('title').show(10)

+--------------------+-----+
|               title|count|
+--------------------+-----+
|$1,000,000 Duck (...|   37|
|'Night Mother (1986)|   70|
|'Til There Was Yo...|   52|
|  'burbs, The (1989)|  303|
|...And Justice fo...|  199|
|        1-900 (1994)|    2|
|10 Things I Hate ...|  700|
|101 Dalmatians (1...|  565|
|101 Dalmatians (1...|  364|
| 12 Angry Men (1957)|  616|
+--------------------+-----+


In [26]:
active_titles_sdf = ratings_by_title_sdf.filter(col('count') >= 250)
active_titles_sdf.sort('title').show(10)

+--------------------+-----+
|               title|count|
+--------------------+-----+
|  'burbs, The (1989)|  303|
|10 Things I Hate ...|  700|
|101 Dalmatians (1...|  565|
|101 Dalmatians (1...|  364|
| 12 Angry Men (1957)|  616|
|13th Warrior, The...|  750|
|2 Days in the Val...|  286|
|20,000 Leagues Un...|  575|
|2001: A Space Ody...| 1716|
|         2010 (1984)|  470|
+--------------------+-----+


In [27]:
active_titles_list = [row.title for row in active_titles_sdf.sort('title').select('title').collect()]
active_titles_list[:5]

["'burbs, The (1989)",
 '10 Things I Hate About You (1999)',
 '101 Dalmatians (1961)',
 '101 Dalmatians (1996)',
 '12 Angry Men (1957)']

In [28]:
mean_ratings_sdf = mean_ratings_pivot.sort('title').filter(col('title').isin(active_titles_list))
mean_ratings_sdf.show(5)

+--------------------+------------------+------------------+
|               title|                 F|                 M|
+--------------------+------------------+------------------+
|  'burbs, The (1989)|2.7934782608695654| 2.962085308056872|
|10 Things I Hate ...| 3.646551724137931|3.3119658119658117|
|101 Dalmatians (1...|3.7914438502673797|               3.5|
|101 Dalmatians (1...|              3.24| 2.911214953271028|
| 12 Angry Men (1957)| 4.184397163120567| 4.328421052631579|
+--------------------+------------------+------------------+


### Show the top list of movies

In [29]:
top_female_ratings_sdf = mean_ratings_sdf.sort(mean_ratings_sdf.F.desc())
top_female_ratings_sdf.show(10)

+--------------------+-----------------+------------------+
|               title|                F|                 M|
+--------------------+-----------------+------------------+
|Close Shave, A (1...|4.644444444444445| 4.473794549266247|
|Wrong Trousers, T...|4.588235294117647| 4.478260869565218|
|Sunset Blvd. (a.k...|4.572649572649572|4.4645892351274785|
|Wallace & Gromit:...|4.563106796116505| 4.385074626865672|
|Schindler's List ...| 4.56260162601626|  4.49141503848431|
|Shawshank Redempt...|4.539074960127592|          4.560625|
|Grand Day Out, A ...|4.537878787878788| 4.293255131964809|
|To Kill a Mocking...|4.536666666666667| 4.372611464968153|
|Creature Comforts...|4.513888888888889| 4.272277227722772|
|Usual Suspects, T...|4.513317191283293| 4.518248175182482|
+--------------------+-----------------+------------------+


In [30]:
top_male_ratings_sdf = mean_ratings_sdf.sort(mean_ratings_sdf.M.desc())
top_male_ratings_sdf.show(10)

+--------------------+------------------+------------------+
|               title|                 F|                 M|
+--------------------+------------------+------------------+
|Godfather, The (1...|4.3146997929606625| 4.583333333333333|
|Seven Samurai (Th...| 4.481132075471698| 4.576628352490421|
|Shawshank Redempt...| 4.539074960127592|          4.560625|
|Raiders of the Lo...|4.3321678321678325| 4.520597322348094|
|Usual Suspects, T...| 4.513317191283293| 4.518248175182482|
|Star Wars: Episod...| 4.302936630602782| 4.495307167235495|
|Schindler's List ...|  4.56260162601626|  4.49141503848431|
|Wrong Trousers, T...| 4.588235294117647| 4.478260869565218|
|Close Shave, A (1...| 4.644444444444445| 4.473794549266247|
|  Rear Window (1954)| 4.484536082474227|4.4729907773386035|
+--------------------+------------------+------------------+


# Measuring Rating Disagreement

Here we want to find the movies that are most different between male and female users. 

In [31]:
mean_ratings_sdf = mean_ratings_sdf.withColumn('diff', mean_ratings_sdf['M'] - mean_ratings_sdf['F'])

Sorting ascending my diff shows us the movies that are rated high by women, but low by men.

In [32]:
mean_ratings_sdf.sort(mean_ratings_sdf.diff.asc()).show(10)

+--------------------+------------------+------------------+-------------------+
|               title|                 F|                 M|               diff|
+--------------------+------------------+------------------+-------------------+
|Dirty Dancing (1987)| 3.790378006872852|  2.95959595959596|-0.8307820472768923|
|Jumpin' Jack Flas...|3.2547169811320753| 2.578358208955224|-0.6763587721768514|
|       Grease (1978)|3.9752650176678443|3.3670411985018727|-0.6082238191659717|
| Little Women (1994)|3.8705882352941177|3.3217391304347825|-0.5488491048593351|
|Steel Magnolias (...| 3.901734104046243|3.3659574468085105|-0.5357766572377325|
|    Anastasia (1997)|               3.8| 3.281609195402299| -0.518390804597701|
|Rocky Horror Pict...| 3.673015873015873|3.1601307189542482|-0.5128851540616246|
|Color Purple, The...|  4.15819209039548| 3.659340659340659|-0.4988514310548209|
|Age of Innocence,...| 3.827067669172932|3.3395061728395063|-0.4875614963334258|
|   Free Willy (1993)|2.9213

Reversing the order shows us the movies that are rated how by men and low by woman. 

In [33]:
mean_ratings_sdf.sort(mean_ratings_sdf.diff.desc()).show(10)

+--------------------+------------------+------------------+------------------+
|               title|                 F|                 M|              diff|
+--------------------+------------------+------------------+------------------+
|Good, The Bad and...| 3.494949494949495|4.2213001383125865|0.7263506433630917|
|Kentucky Fried Mo...| 2.878787878787879|3.5551470588235294|0.6763591800356505|
|Dumb & Dumber (1994)| 2.697986577181208| 3.336594911937378|  0.63860833475617|
|Longest Day, The ...| 3.411764705882353| 4.031446540880503|0.6196818349981501|
|Cable Guy, The (1...|              2.25|2.8637873754152823|0.6137873754152823|
|Evil Dead II (Dea...|3.2972972972972974| 3.909282700421941|0.6119854031246437|
|  Hidden, The (1987)|3.1379310344827585|3.7450980392156863|0.6071670047329278|
|    Rocky III (1982)|2.3617021276595747| 2.943502824858757|0.5818006971991823|
|   Caddyshack (1980)| 3.396135265700483| 3.969736842105263|0.5736015764047799|
|For a Few Dollars...| 3.409090909090909

Finding movies with highest disagreement of viewers independent of gender:
* calculate variance and standard deviation of the ratings
* filter titles with ratings equal or higher 250 (list active_titles from earlier) 
* sort movies in descending order

In [34]:
from pyspark.sql.functions import mean, stddev, col

In [35]:
rating_std_by_title_sdf = data_sdf.groupBy('title').agg(stddev(col('rating')).alias('std'))
rating_std_by_title_sdf.sort('title').show(5)

+--------------------+------------------+
|               title|               std|
+--------------------+------------------+
|$1,000,000 Duck (...| 1.092562901481509|
|'Night Mother (1986)|1.1186356670934383|
|'Til There Was Yo...| 1.020158802295237|
|  'burbs, The (1989)|1.1077601938323738|
|...And Justice fo...|0.8781102494980857|
+--------------------+------------------+


In [36]:
rating_std_by_title_sdf = rating_std_by_title_sdf.filter(col('title').isin(active_titles_list))
rating_std_by_title_sdf.sort(rating_std_by_title_sdf.std.desc()).show(10)

+--------------------+------------------+
|               title|               std|
+--------------------+------------------+
|Dumb & Dumber (1994)|1.3213331436461841|
|Blair Witch Proje...|1.3163677138860728|
|Natural Born Kill...|1.3071976069005837|
|    Tank Girl (1995)| 1.277695392561525|
|Rocky Horror Pict...|1.2601772839374215|
|Eyes Wide Shut (1...| 1.259623523675699|
|        Evita (1996)|1.2536310553940435|
|Billy Madison (1995)| 1.249970159586527|
|Fear and Loathing...|1.2464080767555825|
|Bicentennial Man ...|1.2455325276601854|
+--------------------+------------------+


In [37]:
display(rating_std_by_title_sdf.sort(rating_std_by_title_sdf.std.desc()))

DataFrame[title: string, std: double]